In [ ]:
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication
import os

### Login to OpenShift Cluster

In [ ]:
auth = TokenAuthentication(
    # Get from `oc whoami -t`
    token = "XXXXX",
    # Get from `oc whoami --show-server`
    server = "XXXXX",
    skip_tls=False
)
auth.login()

### Create Ray Cluster

In [ ]:
cluster = Cluster(ClusterConfiguration(
    name='raytest', 
    namespace='fine-tuning', 
    # head_gpus=0, # Head node
    num_gpus=1,
    num_workers=3,
    min_cpus=2,
    max_cpus=2,
    min_memory=8,
    max_memory=8,
    image="quay.io/rhoai/ray:2.23.0-py39-cu121-torch",
    instascale=False, # Enbale instascale or not
    # machine_types=["m5.xlarge", "g4dn.xlarge"], # instascale machine type
))

In [ ]:
# Bring up the Ray cluster
cluster.up()

In [ ]:
# Check cluster status
cluster.status()

In [ ]:
# Wait for Ray cluster to be ready
cluster.wait_ready()

In [ ]:
# Check cluster details
cluster.details()

### Ray Job Submission

In [28]:
client = cluster.job_client

In [ ]:
submission_id = client.submit_job(
    entrypoint="python ray_fine_tuning.py",
    runtime_env={
        "env_vars": {
            "AWS_ACCESS_KEY_ID": os.environ.get('AWS_ACCESS_KEY_ID'),
            "AWS_SECRET_ACCESS_KEY": os.environ.get('AWS_SECRET_ACCESS_KEY'),
            "AWS_S3_ENDPOINT": os.environ.get('AWS_S3_ENDPOINT')
        },
        "pip": "requirements.txt",
        "working_dir": "./",
        "excludes": ["*.ipynb", "*.md"]
    },
)
print(submission_id)

### Clean up

In [ ]:
client.stop_job(submission_id)

In [ ]:
client.delete_job(submission_id)

In [ ]:
cluster.down()